In [9]:
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator)
from data_generation import generate_input_data
from model_nile import ModelNile
import random
from ema_workbench.em_framework.optimization import (
    EpsilonProgress,
    ArchiveLogger,
)

nile_model = ModelNile()
nile_model = generate_input_data(nile_model, sim_horizon=20)

#model
em_model = Model("NileProblem", function=nile_model)

#levers - RBF parameters
parameter_count = nile_model.overarching_policy.get_total_parameter_count()
n_inputs = nile_model.overarching_policy.functions["release"].n_inputs
n_outputs = nile_model.overarching_policy.functions["release"].n_outputs
RBF_count = nile_model.overarching_policy.functions["release"].RBF_count
p_per_RBF = 2 * n_inputs + n_outputs

lever_list = list()
for i in range(parameter_count):
    modulus = (i - n_outputs) % p_per_RBF
    if (
        (i >= n_outputs)
        and (modulus < (p_per_RBF - n_outputs))
        and (modulus % 2 == 0)
    ):  # centers:
        lever_list.append(RealParameter(f"v{i}", -1, 1))
    else:  # linear parameters for each release, radii and weights of RBFs:
        lever_list.append(RealParameter(f"v{i}", 0, 1))

em_model.levers = lever_list



#outcomes
em_model.outcomes = [
        ScalarOutcome("egypt_irr", ScalarOutcome.MINIMIZE),
        ScalarOutcome("egypt_90", ScalarOutcome.MINIMIZE),
        ScalarOutcome("egypt_low_had", ScalarOutcome.MINIMIZE),
        ScalarOutcome("sudan_irr", ScalarOutcome.MINIMIZE),
        ScalarOutcome("sudan_90", ScalarOutcome.MINIMIZE),
        ScalarOutcome("ethiopia_hydro", ScalarOutcome.MAXIMIZE),
    ]

convergence_metrics = [
        EpsilonProgress(),
        ArchiveLogger(
            "archive_logs",
            [lever.name for lever in em_model.levers],
            [outcome.name for outcome in em_model.outcomes],
        ),
    ]

In [11]:
epsilon_list = [1e-1, 1e-2, 1e-2, 1e-1, 1e-2, 1e-1]


with MultiprocessingEvaluator(em_model) as evaluator:
        results, convergence = evaluator.optimize(
            nfe=5e3,
            searchover="levers",
            epsilons=epsilon_list,
            convergence_freq=500,
            convergence=convergence_metrics,
        )

  0%|                                                 | 0/5000 [00:00<?, ?it/s]interpolation can only be 'linear', 'lower' 'higher', 'midpoint', or 'nearest'
Traceback (most recent call last):
  File "/Users/zuzannaosika/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "/Users/zuzannaosika/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/zuzannaosika/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "/Users/zuzannaosika/miniforge3/lib/python3.9/site-packages/ema_workbench/util/ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "/Users/zuzannaosika/miniforge3/lib/python3.9/site-packages/ema_workbench/em_framework/model.py", line 403, in run_experiment
    mo

KeyboardInterrupt: 